# Design Patterns

## 1 Design patterns

Object-oriented design facilitates reusable, robust, and adaptable software. 

Computing researchers and practitioners have developed a variety of organizational concepts and methodologies for designing quality object-oriented software that is concise, correct, and reusable. 

Of special relevance to our course is the concept of a **design pattern**.

In software engineering, a software design pattern is a general, reusable solution to a commonly occurring problem within a given context in software design. 

It is not a finished design that can be transformed directly into source or machine code. Rather, it is **a description or template** for how to solve a problem that can be used in many different situations.

Design patterns are **formalized** best practices that the programmer can use to solve common problems when designing an application or system.

Design patterns became a popular topic in late 90s after the so-called Gang of Four (GoF: Gamma, Helm, Johson, and Vlissides) published their book Design Patterns: `Elements of Reusable Object-Oriented Software`

The book describes design patterns as a core design solution to reoccurring problems in software and classifies each design pattern into categories according to the nature of the problem. 

* Each pattern is given `a name`, `a problem description`it is trying to solve, `a design solution` that the design pattern supplies, and `an explanation of the consequences of using it`

Some particular design patterns are so important that they are built into Python/C++.



**For Example:Object-oriented design patterns**

**Object-oriented design patterns** typically show relationships and interactions between `classes` or `objects`, without specifying the final application classes or objects that are involved. 

**Encapsulation (data hiding)**

* **Problem**: Exposed fields can be directly manipulated from outside, leading to violations of the representation invariant or undesirable dependences that prevent changing the
implementation.

* **Solution:** Hide some components, permitting only stylized access to the object.
    
* **Disadvantages**: The interface may not (efficiently) provide all desired operations. Indirection may reduce performance.


**Subclassing (inheritance)**

* **Problem:** Similar abstractions have similar members (fields and methods). Repeating these is tedious, error-prone, and a maintenance headache.
    
* **Solution:** Inherit default members from a superclass; select the correct implementation via run-time dispatching.

* **Disadvantages:** Code for a class is spread out, potentially reducing understandability. Run-time dispatching introduces overhead.

Other design patterns are so important that they are built into other languages. Some design patterns may never be built into languages, but are still useful in their place

**When (not) to use design patterns**

A software designer must trade off the advantages against the disadvantages when deciding whether to use a design pattern. **Tradeoffs**
between flexibility and performance are common, as you will often discover in computer science (and other fields).

The first rule of design patterns is the same as the first rule of `optimization`: **delay**. 

* Just as you shouldn’t optimize prematurely, don’t use design patterns prematurely.

It may be best to **first** implement something and **ensure that it works**, then use the `design pattern` to `improve` weaknesses;

* this is especially true if `you do not yet grasp all the details of the design`. (If you fully understand the domain and problem, it may make sense to use design patterns from the start, just as it makes sense to use a more efficient rather than a less efficient algorithm from the very beginning in some applications.)


Design patterns may `increase` or `decrease` the **understandability** of a design or implementation.


* They can `decrease` understandability by adding `indirection or increasing` the amount of `code.`

* They can `increase` understandability by improving `modularity`, better `separating` concerns, and easing `description`. 

Once you learn the vocabulary of design patterns, you will be able to communicate more precisely and rapidly with other people who know the vocabulary. 

* It’s much better to say,**“This is an instance of the `visitor pattern`”** than “This is some code that traverses a structure and makes callbacks, and some certain methods must be present, and they are called in this particular way and in this particular order.”


Most people use design patterns when they notice a problem with their design—something that ought to be easy isn’t—or their implementation —such as performance. 

* Examine the offending design or code. What are its problems, and what compromises does it make? What would you like to do that is presently too hard? Then, check a design pattern reference. Look for patterns that address the issues you are concerned with

## 2 Example: Factory Pattern

Factory Method is a creational design pattern used to create concrete implementations of a common interface.

It separates the process of creating an object from the code that depends on the interface of the object.

For example, an application requires an object with a specific interface to perform its tasks. The concrete implementation of the interface is **identified** by some **parameter**.

Instead of using a complex `if/elif/else` conditional structure to determine the concrete implementation, the application delegates that decision to a separate component that creates the concrete object. With this approach, the application code is simplified, making it more reusable and easier to maintain.

* [Factory Pattern in SimVCCE](https://github.com/PySEE/SimVCCE)

**For Example in Python:**


* **`../../SimVCCE/vccpython`**

set `../../SimVCCE/vccpython` as the working dir

In [14]:
%cd ../../SimVCCE/vccpython

J:\SEU\SEE\PySEE\SimVCCE\vccpython


In [7]:
%pwd

'J:\\SEU\\SEE\\PySEE\\SimVCCE\\vccpython'

###  The device class

We write a class to represent a  vapor-compression refrigeration cycle. A cycle consists of many devices(among other objects, perhaps).

**Components Package**

./components/

* `__init__.py`

* node.py

* compressor.py

* condenser.py

* expansionvalve.py

* evaporator.py

In [ ]:
# %load ./components/compressor.py

"""
The Object-oriented Programming Demo of VCR Cycle
  Compressor:Isentropic compression (ideal VCR cycle)
"""
from .node import *


class Compressor:
    """ Isentropic compression of the refrigerant"""
    energy = "CompressionWork"
    devtype = "COMPRESSOR"

    def __init__(self, dictDev, nodes):
        """
        Initializes 
        """
        self.name = dictDev['name']
        self.iNode = nodes[dictDev['iNode']]
        self.oNode = nodes[dictDev['oNode']]
        self.Wc = None

    def state(self):
        """
        Isentropic compression (ideal VCR cycle)
        """
        self.oNode.s = self.iNode.s

    def balance(self):
        """  mass and energy balance    """
        # mass balance
        if self.iNode.mdot is not None:
            self.oNode.mdot = self.iNode.mdot
        elif self.oNode.mdot is not None:
            self.iNode.mdot = self.oNode.mdot
        # energy
        self.Wc = self.iNode.mdot * (self.oNode.h - self.iNode.h)

    def __str__(self):
        result = '\n' + self.name
        result += '\n' + Node.title
        result += '\n' + self.iNode.__str__()
        result += '\n' + self.oNode.__str__()
        result += '\nWc(kW): \t{:>.2f}'.format(self.Wc)
        return result


###  The Creator of VCR Cycle

* To create the instance of each component

In [ ]:
# %load ./vcc/vccobj.py
"""
The Object-oriented Programming Demo of VCR Cycle 

VCRCycle: the Simulator class of VCR Cycle  
"""

import time
import getpass
from platform import os

from components.node import Node
from components import compdict


class VCRCycle:

    def __init__(self, dictcycle):
        """
          dictcycle={"name":namestring,
                     "nodes":[{node1},{node2},...],
                     "comps":[{component1},{component2},...]
                     }
          TO:           
             self.nodes : dict of all node objects:
                {node id:node object}
             self.comps : dict of all component objects:
                {device name:device obiect}
        """
        self.name = dictcycle["name"]
        dictnodes = dictcycle["nodes"]
        dictcomps = dictcycle["comps"]

        # 1 convert dict to the dict of node objects {node id:node object}
        self.nodes = {}
        for curnode in dictnodes:
            self.nodes[curnode["id"]] = Node(curnode)

        # 2 convert dict to the dict of device objects: {device name:device obiect}
        self.comps = {}
        for curdev in dictcomps:
            self.comps[curdev['name']] = compdict[curdev['devtype']](
                curdev, self.nodes)

    def ComponentState(self):
        for key in self.comps:
            self.comps[key].state()

        for key in self.nodes:
            if self.nodes[key].stateok == False:
                self.nodes[key].state()

    def ComponentBalance(self):
        for curdev in self.comps:
            self.comps[curdev].balance()

    def simulator(self):
        self.ComponentState()
        self.ComponentBalance()

        self.Wc = 0
        self.Qlow = 0

        for key in self.comps:
            if self.comps[key].energy == "CompressionWork":
                self.Wc += self.comps[key].Wc
            elif self.comps[key].energy == "RefrigerationCapacity":
                self.Qlow += self.comps[key].Qlow

        self.cop = self.Qlow / self.Wc
        self.Qlow = self.Qlow*60*(1/211)

    def __setformatstr(self, formatstr, result):
        result += formatstr.format('Compression Work(kW): ', self.Wc)
        result += formatstr.format('Refrigeration Capacity(ton): ', self.Qlow)
        result += formatstr.format('The coefficient of performance: ', self.cop)
        return result

    def __str__(self):
        str_curtime = time.strftime(
            "%Y/%m/%d %H:%M:%S", time.localtime(time.time()))
        result = "\nRefrigeration Cycle: {} (Time: {} by {} on {})\n".format(
            self.name, str_curtime,getpass.getuser(),os.popen('hostname').read())
        try:
            formatstr = "{:>35} {:>6.3f}\n"
            result = self.__setformatstr(formatstr, result)
        except:
            formatstr = "{} {}\n"
            result = self.__setformatstr(formatstr, result)
        return result



The each componet instance is created  by **HARD CODE** that is tedious ,less flexible,low understandability/performance.. 

There must be a better way. The **Factory design patterns** provide an answer.

### The Factory Pattern

* `./components/__init__.py`

compdict(jump table跳转表)

* key:value-> Type String: class  name

```python
compdict = {
    "COMPRESSOR": Compressor,
    "CONDENSER": Condenser,
    "EXPANSIONVALVE": ExpansionValve,
    "EVAPORATOR": Evaporator
}


```
* `./vcc/cycleobj.py`

```python
 # 2 convert dict to the dict of device objects: {device name:device object}
        self.comps = {}
        for curdev in dictcomps:
            self.comps[curdev['name']] = compdict[curdev['devtype']](curdev, self.nodes)
```

In [ ]:
# %load ./components/__init__.py
"""
The Object-oriented Programming Demo of VCR Cycle 

    Components Package  
"""

from .node import Node

from .compressor import Compressor
from .condenser import Condenser
from .expansionvalve import ExpansionValve
from .evaporator import Evaporator


# ------------------------------------------------------------------------------
# compdict(jump table)
#  1: key:value-> Type String: class  name
#  2    add the new key:value to the dict after you add the new device class/type
# --------------------------------------------------------------------------------

compdict = {
    "COMPRESSOR": Compressor,
    "CONDENSER": Condenser,
    "EXPANSIONVALVE": ExpansionValve,
    "EVAPORATOR": Evaporator
}


In [ ]:
# %load ./vcc/vccobj.py
"""
The Object-oriented Programming Demo of VCR Cycle 

VCRCycle: the Simulator class of VCR Cycle  
"""

import time
import getpass
from platform import os

from components.node import Node
from components import compdict


class VCRCycle:

    def __init__(self, dictcycle):
        """
          dictcycle={"name":namestring,
                     "nodes":[{node1},{node2},...],
                     "comps":[{component1},{component2},...]
                     }
          TO:           
             self.nodes : dict of all node objects:
                {node id:node object}
             self.comps : dict of all component objects:
                {device name:device obiect}
        """
        self.name = dictcycle["name"]
        dictnodes = dictcycle["nodes"]
        dictcomps = dictcycle["comps"]

        # 1 convert dict to the dict of node objects {node id:node object}
        self.nodes = {}
        for curnode in dictnodes:
            self.nodes[curnode["id"]] = Node(curnode)

        # 2 convert dict to the dict of device objects: {device name:device obiect}
        self.comps = {}
        for curdev in dictcomps:
            self.comps[curdev['name']] = compdict[curdev['devtype']](
                curdev, self.nodes)

    def ComponentState(self):
        for key in self.comps:
            self.comps[key].state()

        for key in self.nodes:
            if self.nodes[key].stateok == False:
                self.nodes[key].state()

    def ComponentBalance(self):
        for curdev in self.comps:
            self.comps[curdev].balance()

    def simulator(self):
        self.ComponentState()
        self.ComponentBalance()

        self.Wc = 0
        self.Qlow = 0

        for key in self.comps:
            if self.comps[key].energy == "CompressionWork":
                self.Wc += self.comps[key].Wc
            elif self.comps[key].energy == "RefrigerationCapacity":
                self.Qlow += self.comps[key].Qlow

        self.cop = self.Qlow / self.Wc
        self.Qlow = self.Qlow*60*(1/211)

    def __setformatstr(self, formatstr, result):
        result += formatstr.format('Compression Work(kW): ', self.Wc)
        result += formatstr.format('Refrigeration Capacity(ton): ', self.Qlow)
        result += formatstr.format('The coefficient of performance: ', self.cop)
        return result

    def __str__(self):
        str_curtime = time.strftime(
            "%Y/%m/%d %H:%M:%S", time.localtime(time.time()))
        result = "\nRefrigeration Cycle: {} (Time: {} by {} on {})\n".format(
            self.name, str_curtime,getpass.getuser(),os.popen('hostname').read())
        try:
            formatstr = "{:>35} {:>6.3f}\n"
            result = self.__setformatstr(formatstr, result)
        except:
            formatstr = "{} {}\n"
            result = self.__setformatstr(formatstr, result)
        return result
